In [4]:
# imports
from tensorflow import keras
from keras import layers

import numpy as np

import sente
from sente import sgf

Initialize the network

Network Architecture originally based on The original AlphaGo Lee Architecture

> "The input to the policy network is a 19 × 19 × 48 image stack consisting of 48 feature planes. The first
> hidden layer zero pads the input into a 23 × 23 image, then convolves k filters of kernel size 5 × 5 with
> stride 1 with the input image and applies a rectifier nonlinearity. Each of the subsequent hidden layers 2 to
> 12 zero pads the respective previous hidden layer into a 21 × 21 image, then convolves k filters of kernel size
> 3 × 3 with stride 1, again followed by a rectifier nonlinearity. The final layer convolves 1 filter of kernel
> size 1 × 1 with stride 1, with a different bias for each position, and applies a softmax func- tion. The match
> version of AlphaGo used k = 192 filters; Fig. 2b and Extended Data Table 3 additionally show the results of
> training with k = 128, 256 and 384 filters."

A few modifications have been made

- Only has 4 input Features instead of 48
- Uses batch normalization instead of dropout

In [11]:
# get the numpy spec from a generic game
input_numpy = sente.Game().numpy()

# input layer
input_layer = layers.Input(shape=input_numpy.shape)

# First layer has a kernel size of 5
x = layers.Conv2D(filters=192,
                  kernel_size=5,
                  padding="same",
                  activation="relu"
                  )(input_layer)
x = layers.BatchNormalization()(x)

# subsequent layers have kernel sizes of 3
for i in range(11):
    x = layers.Conv2D(filters=192,
                      kernel_size=3,
                      activation="relu",
                      padding="same")(x)
    x = layers.BatchNormalization()(x)

x = layers.Conv2D(filters=1,
                  kernel_size=1)(x)
output = layers.Softmax()(x)

model = keras.Model(inputs=input_layer, outputs=output, name="Policy Network")
model.summary()

Model: "Policy Network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 19, 19, 4)]       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 19, 19, 192)       19392     
                                                                 
 batch_normalization_24 (Bat  (None, 19, 19, 192)      768       
 chNormalization)                                                
                                                                 
 conv2d_26 (Conv2D)          (None, 19, 19, 192)       331968    
                                                                 
 batch_normalization_25 (Bat  (None, 19, 19, 192)      768       
 chNormalization)                                                
                                                                 
 conv2d_27 (Conv2D)          (None, 19, 19, 192)    